In [ ]:
!python -m pip install pandas requests tabulate

In [ ]:
import json
import pandas as pd
import requests
import re

In [ ]:
def curl_json(url):
    return json.loads(requests.get(f"https://ddragon.leagueoflegends.com{url}").content)


versions = curl_json("/api/versions.json")
last_version = versions[0]
champion = curl_json(f"/cdn/{last_version}/data/en_US/champion.json")
names = [v["name"] for k, v in champion["data"].items()]
names.sort()
print(f"version: {last_version}\n{len(names)} champions")


In [ ]:
def shortcuts_generator(s):
    ls = len(s)
    for i in range(1, 2 ** len(s)):
        mask = bin(i)[2:].rjust(ls, '0')
        shortcut = "".join([si for i, si in enumerate(s) if mask[i] == "1"])
        yield shortcut


names_shortcuts = [(name, set(shortcuts_generator(name.lower().replace(" ", "")))) for name in names]

shortest_shortcuts = []

for name, shortcuts in names_shortcuts:
    copy = list(names_shortcuts)
    copy.remove((name, shortcuts))
    all_others = set()
    for n, s in copy:
        all_others |= s
    shortcut = list(shortcuts - all_others)
    if not shortcut:
        shortest_shortcuts.append((name, ""))
        continue
    min_len = min([len(s) for s in shortcut])
    shortest_shortcut = [s for s in shortcut if len(s) == min_len]
    shortest_shortcut.sort()
    shortest_shortcuts.append((name, ", ".join(shortest_shortcut)))

In [ ]:
df = pd.DataFrame(shortest_shortcuts, columns=["Champions", "Champion select shortcuts"])
table = df.to_markdown(index=False)
table = re.sub("\\s{2,}", " ", table)
table = re.sub("-{2,}", "-", table)
open("README.md", "w").write(table)